# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [209]:
problem = '''Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets. 

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs).'''

## 2. Ask for parameters

In [210]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [211]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Decision Variables:

*  *x*<sub>1</sub>: Acres of land dedicated to wheat production.
*  *x*<sub>2</sub>: Acres of land dedicated to corn production.
*  *x*<sub>3</sub>: Acres of land dedicated to sugar beet production.

## Auxiliary Variables:

* *w*<sub>buy</sub>: Tons of wheat purchased.
* *c*<sub>buy</sub>: Tons of corn purchased.
* *w*<sub>sell</sub>: Tons of wheat sold.
* *c*<sub>sell</sub>: Tons of corn sold.
* *s*<sub>q</sub>: Tons of sugar beets sold at the quota price.
* *s*<sub>e</sub>: Tons of sugar beets sold in excess of the quota. 


# 2. Ask for objective

In [212]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [213]:
Markdown(response2.text)

Minimize: 

 $$150x_1 + 230x_2 + 260x_3  + 204w_{buy} + 180c_{buy} - 170w_{sell} - 150c_{sell} - 36s_q - 10s_e$$ 


# 3. Ask for constraints

In [214]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [215]:
Markdown(response3.text)

## Constraints:

**Land availability:**

*  *x*<sub>1</sub> + *x*<sub>2</sub> + *x*<sub>3</sub> ≤ 500 

**Production and purchase balance:**

* 2.5*x*<sub>1</sub> + *w*<sub>buy</sub> - *w*<sub>sell</sub> ≥ 200  
* 3*x*<sub>2</sub> + *c*<sub>buy</sub> - *c*<sub>sell</sub> ≥ 240 

**Sugar beet quota:**

* *s*<sub>q</sub> ≤ 6000

**Sugar beet production:**

* 20*x*<sub>3</sub> = *s*<sub>q</sub> + *s*<sub>e</sub> 

**Non-negativity:**

* *x*<sub>1</sub>, *x*<sub>2</sub>, *x*<sub>3</sub>, *w*<sub>buy</sub>, *c*<sub>buy</sub>, *w*<sub>sell</sub>, *c*<sub>sell</sub>, *s*<sub>q</sub>, *s*<sub>e</sub> ≥ 0 


## 3. Generate the pyomo code

In [216]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [217]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
land_availability = 500  
wheat_demand = 200
corn_demand = 240
sugar_beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of wheat
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of corn
model.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of sugar beet

# Auxiliary Variables
model.w_buy = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat purchased
model.c_buy = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn purchased
model.w_sell = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_sell = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn sold
model.s_q = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold at quota price
model.s_e = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold in excess

# Objective Function
model.profit = pyo.Objective(
    expr=150 * model.x1
    + 230 * model.x2
    + 260 * model.x3
    + 204 * model.w_buy
    + 180 * model.c_buy
    - 170 * model.w_sell
    - 150 * model.c_sell
    - 36 * model.s_q
    - 10 * model.s_e,
    sense=pyo.minimize,
)

# Constraints
model.land_constraint = pyo.Constraint(
    expr=model.x1 + model.x2 + model.x3 <= land_availability
)

model.wheat_balance = pyo.Constraint(
    expr=2.5 * model.x1 + model.w_buy - model.w_sell >= wheat_demand
)

model.corn_balance = pyo.Constraint(
    expr=3 * model.x2 + model.c_buy - model.c_sell >= corn_demand
)

model.sugar_quota = pyo.Constraint(expr=model.s_q <= sugar_beet_quota)

model.sugar_production = pyo.Constraint(
    expr=20 * model.x3 == model.s_q + model.s_e
)

# Solve
solver = pyo.SolverFactory("glpk")
solver.solve(model)

# Print the solution
print("Solution:")
print(f"Wheat Production: {pyo.value(model.x1)} acres")
print(f"Corn Production: {pyo.value(model.x2)} acres")
print(f"Sugar Beet Production: {pyo.value(model.x3)} acres")
print(f"Wheat Purchased: {pyo.value(model.w_buy)} tons")
print(f"Corn Purchased: {pyo.value(model.c_buy)} tons")
print(f"Wheat Sold: {pyo.value(model.w_sell)} tons")
print(f"Corn Sold: {pyo.value(model.c_sell)} tons")
print(f"Sugar Beets Sold (Quota): {pyo.value(model.s_q)} tons")
print(f"Sugar Beets Sold (Excess): {pyo.value(model.s_e)} tons")
print(f"Total Profit: ${pyo.value(model.profit)}")

```


## 4. Input problem data and try running the generated code

In [218]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
land_availability = 500  
wheat_demand = 200
corn_demand = 240
sugar_beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of wheat
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of corn
model.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of sugar beet

# Auxiliary Variables
model.w_buy = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat purchased
model.c_buy = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn purchased
model.w_sell = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_sell = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn sold
model.s_q = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold at quota price
model.s_e = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold in excess

# Objective Function
model.profit = pyo.Objective(
    expr=150 * model.x1
    + 230 * model.x2
    + 260 * model.x3
    + 204 * model.w_buy
    + 180 * model.c_buy
    - 170 * model.w_sell
    - 150 * model.c_sell
    - 36 * model.s_q
    - 10 * model.s_e,
    sense=pyo.minimize,
)

# Constraints
model.land_constraint = pyo.Constraint(
    expr=model.x1 + model.x2 + model.x3 <= land_availability
)

model.wheat_balance = pyo.Constraint(
    expr=2.5 * model.x1 + model.w_buy - model.w_sell >= wheat_demand
)

model.corn_balance = pyo.Constraint(
    expr=3 * model.x2 + model.c_buy - model.c_sell >= corn_demand
)

model.sugar_quota = pyo.Constraint(expr=model.s_q <= sugar_beet_quota)

model.sugar_production = pyo.Constraint(
    expr=20 * model.x3 == model.s_q + model.s_e
)

# Solve
solver = pyo.SolverFactory("glpk")
solver.solve(model)

# Print the solution
print("Solution:")
print(f"Wheat Production: {pyo.value(model.x1)} acres")
print(f"Corn Production: {pyo.value(model.x2)} acres")
print(f"Sugar Beet Production: {pyo.value(model.x3)} acres")
print(f"Wheat Purchased: {pyo.value(model.w_buy)} tons")
print(f"Corn Purchased: {pyo.value(model.c_buy)} tons")
print(f"Wheat Sold: {pyo.value(model.w_sell)} tons")
print(f"Corn Sold: {pyo.value(model.c_sell)} tons")
print(f"Sugar Beets Sold (Quota): {pyo.value(model.s_q)} tons")
print(f"Sugar Beets Sold (Excess): {pyo.value(model.s_e)} tons")
print(f"Total Profit: ${pyo.value(model.profit)}")

Solution:
Wheat Production: 120.0 acres
Corn Production: 80.0 acres
Sugar Beet Production: 300.0 acres
Wheat Purchased: 0.0 tons
Corn Purchased: 0.0 tons
Wheat Sold: 99.9999999999999 tons
Corn Sold: 0.0 tons
Sugar Beets Sold (Quota): 6000.0 tons
Sugar Beets Sold (Excess): 0.0 tons
Total Profit: $-118599.99999999999


## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [219]:
print(response.text)

## Decision Variables:

*  *x*<sub>1</sub>: Acres of land dedicated to wheat production.
*  *x*<sub>2</sub>: Acres of land dedicated to corn production.
*  *x*<sub>3</sub>: Acres of land dedicated to sugar beet production.

## Auxiliary Variables:

* *w*<sub>buy</sub>: Tons of wheat purchased.
* *c*<sub>buy</sub>: Tons of corn purchased.
* *w*<sub>sell</sub>: Tons of wheat sold.
* *c*<sub>sell</sub>: Tons of corn sold.
* *s*<sub>q</sub>: Tons of sugar beets sold at the quota price.
* *s*<sub>e</sub>: Tons of sugar beets sold in excess of the quota. 



In [220]:
print(response2.text)

Minimize: 

 $$150x_1 + 230x_2 + 260x_3  + 204w_{buy} + 180c_{buy} - 170w_{sell} - 150c_{sell} - 36s_q - 10s_e$$ 



In [221]:
print(response3.text)

## Constraints:

**Land availability:**

*  *x*<sub>1</sub> + *x*<sub>2</sub> + *x*<sub>3</sub> ≤ 500 

**Production and purchase balance:**

* 2.5*x*<sub>1</sub> + *w*<sub>buy</sub> - *w*<sub>sell</sub> ≥ 200  
* 3*x*<sub>2</sub> + *c*<sub>buy</sub> - *c*<sub>sell</sub> ≥ 240 

**Sugar beet quota:**

* *s*<sub>q</sub> ≤ 6000

**Sugar beet production:**

* 20*x*<sub>3</sub> = *s*<sub>q</sub> + *s*<sub>e</sub> 

**Non-negativity:**

* *x*<sub>1</sub>, *x*<sub>2</sub>, *x*<sub>3</sub>, *w*<sub>buy</sub>, *c*<sub>buy</sub>, *w*<sub>sell</sub>, *c*<sub>sell</sub>, *s*<sub>q</sub>, *s*<sub>e</sub> ≥ 0 



In [222]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
land_availability = 500  
wheat_demand = 200
corn_demand = 240
sugar_beet_quota = 6000

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.x1 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of wheat
model.x2 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of corn
model.x3 = pyo.Var(within=pyo.NonNegativeReals)  # Acres of sugar beet

# Auxiliary Variables
model.w_buy = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat purchased
model.c_buy = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn purchased
model.w_sell = pyo.Var(within=pyo.NonNegativeReals)  # Tons of wheat sold
model.c_sell = pyo.Var(within=pyo.NonNegativeReals)  # Tons of corn sold
model.s_q = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold at quota price
model.s_e = pyo.Var(within=pyo.NonNegativeReals)  # Tons of sugar beets sold in excess

# Objective Function
model.profit = pyo.Objective(
   